In [1]:
!git clone https://github.com/LJOVO/TranSalNet.git

%cd TranSalNet

Cloning into 'TranSalNet'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 58 (delta 9), reused 23 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 3.14 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/TranSalNet


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/Saliency4asd.zip'
extract_dir = 'datasets/Saliency/'
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Unzipped: {zip_file_path} to {extract_dir}")

Unzipped: /content/drive/MyDrive/Saliency4asd.zip to datasets/Saliency/


In [4]:
import shutil

source_folder = '/content/drive/MyDrive/Saliency/models/TransalNet'
destination_folder = 'pretrained_models'

for filename in os.listdir(source_folder):
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)
    shutil.copy(source_path, destination_path)

In [5]:
import os
import torch
import cv2
import numpy as np
from torchvision import transforms, utils, models
import torch.nn as nn
from tqdm import tqdm
from utils.data_process import preprocess_img, postprocess_img
from PIL import Image
from TranSalNet_Res import TranSalNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
toPIL = transforms.ToPILImage()

model = TranSalNet()
model.load_state_dict(torch.load(r'pretrained_models/TranSalNet_Res.pth'))
model = model.to(device)
model.eval()

TranSalNet(
  (encoder): _Encoder(
    (encoder): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
      

In [6]:
#@title For all images

import glob

# Define the input folder and output folder
input_folder = r'datasets/Saliency/Saliency4asd/Images'
output_folder = r'output/TranSalNet'
os.makedirs(output_folder, exist_ok=True)

extensions = ("*.jpg", "*.jpeg", "*.png")
image_paths = []
for ext in extensions:
    image_paths.extend(glob.glob(os.path.join(input_folder, ext)))

for img_path in image_paths:
    print("Processing:", img_path)

    # Preprocess the image
    img = preprocess_img(img_path)
    img = np.array(img) / 255.0
    img = np.expand_dims(np.transpose(img, (2, 0, 1)), axis=0)
    img = torch.from_numpy(img)
    img = img.type(torch.cuda.FloatTensor).to(device)

    # Predicted saliency map
    pred_saliency = model(img)

    pic = toPIL(pred_saliency.squeeze())
    pred_saliency = postprocess_img(pic, img_path)

    filename = os.path.basename(img_path)
    output_path = os.path.join(output_folder, filename)
    ext = os.path.splitext(filename)[1].lower()
    if ext in ['.jpg', '.jpeg']:
        cv2.imwrite(output_path, pred_saliency, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    else:
        cv2.imwrite(output_path, pred_saliency)

    print('Finished processing, saved result at:', output_path)

print("All images processed. Check the output folder:", output_folder)


Processing: datasets/Saliency/Saliency4asd/Images/298.png
Finished processing, saved result at: output/TranSalNet/298.png
Processing: datasets/Saliency/Saliency4asd/Images/171.png
Finished processing, saved result at: output/TranSalNet/171.png
Processing: datasets/Saliency/Saliency4asd/Images/117.png
Finished processing, saved result at: output/TranSalNet/117.png
Processing: datasets/Saliency/Saliency4asd/Images/32.png
Finished processing, saved result at: output/TranSalNet/32.png
Processing: datasets/Saliency/Saliency4asd/Images/82.png
Finished processing, saved result at: output/TranSalNet/82.png
Processing: datasets/Saliency/Saliency4asd/Images/20.png
Finished processing, saved result at: output/TranSalNet/20.png
Processing: datasets/Saliency/Saliency4asd/Images/83.png
Finished processing, saved result at: output/TranSalNet/83.png
Processing: datasets/Saliency/Saliency4asd/Images/110.png
Finished processing, saved result at: output/TranSalNet/110.png
Processing: datasets/Saliency/Sa

In [7]:
#@title download the salicency map obtained from SalFBNet model

from google.colab import files
import shutil
import os

temp_dir = 'temp_download'
os.makedirs(temp_dir, exist_ok=True)

!zip -r "{temp_dir}/TranSalNet.zip" "output/TranSalNet"
shutil.move(f"{temp_dir}/TranSalNet.zip", "TranSalNet.zip")

shutil.rmtree(temp_dir)

# Download the zip file
files.download('TranSalNet.zip')

  adding: output/TranSalNet/ (stored 0%)
  adding: output/TranSalNet/298.png (deflated 18%)
  adding: output/TranSalNet/171.png (deflated 16%)
  adding: output/TranSalNet/117.png (deflated 15%)
  adding: output/TranSalNet/32.png (deflated 23%)
  adding: output/TranSalNet/82.png (deflated 18%)
  adding: output/TranSalNet/20.png (deflated 15%)
  adding: output/TranSalNet/83.png (deflated 14%)
  adding: output/TranSalNet/110.png (deflated 17%)
  adding: output/TranSalNet/129.png (deflated 18%)
  adding: output/TranSalNet/178.png (deflated 16%)
  adding: output/TranSalNet/85.png (deflated 15%)
  adding: output/TranSalNet/63.png (deflated 14%)
  adding: output/TranSalNet/187.png (deflated 19%)
  adding: output/TranSalNet/2.png (deflated 17%)
  adding: output/TranSalNet/79.png (deflated 19%)
  adding: output/TranSalNet/202.png (deflated 14%)
  adding: output/TranSalNet/52.png (deflated 17%)
  adding: output/TranSalNet/5.png (deflated 15%)
  adding: output/TranSalNet/44.png (deflated 19%)
  a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>